# Electron Drift Velocity - Individual Analysis

Much of the analysis involves converting the data to velocity vs $E/P$ and making plots.

Consult the documentation for different Python packages.  Also recommended: the [Whirlwind Tour of Python](https://jakevdp.github.io/WhirlwindTourOfPython/) and the [Python Data Science Handbook] (https://jakevdp.github.io/PythonDataScienceHandbook/) both by Jake VanderPlas.

In [ ]:
import numpy as np  # import the numpy library functions.
import scipy.constants as const # import physical constants
import scipy.interpolate as interp # interpolation functions, used to match position 1 and position 2 measurements
import matplotlib.pyplot as plt # plotting functions
import uncertainties as unc # Uncertainties package.  Good for simple error propagation
# directive below puts the plots in the notebook
%matplotlib inline

## Outline

### Calibration

1. Enter your calibration data set (time values from the digital pulser versus channel numbers on the MCA) and create a line fit and calibration function.
2. Read in your pulse-height analysis results from spreadsheets obtained during data collection.  These should contain at least three columns: (1) Applied accelerating-cage voltage, (2) peak position in channels, and (3) peak width in channels.  Recommended: use the Pandas `read_csv()` or `read_excel()` functions.  You should have a total of 4 spreadsheets: data from position 1 for the 93%-Ar/7%-CO<sub>2</sub> mixture, data from position 2 for the same  mixture, data from position 1 for the 80%-Ar/20%-CO<sub>2</sub> mixture, and data from position 2 for that mixture.
3. Apply your calibration to the peak-position and peak-width data to convert each to time units (microseconds recommended).

### Data reduction
1. Record the necessary physical quantities needed to convert voltage to electric field divided by pressure (typically in V/cm-torr) $E/P$ and, for comparison to Zhao, et al., to electric field (at 1 atmosphere pressure) in V/cm. (You  need dimensions of the accelerating cage.) Also record the necessary quantities to convert your time measurements to velocity values in centimeters per microseconds. (You need a distance between position 1 and position 2).
2. Create a function that allows you to interpolate between measured HV values for position 2 to obtain the estimated time at any voltage.  (This is necessary unless you have taken measurements at position 2 corresponding to every measurement taken for position 1.)  Recommended: use `scipy.interpolate.interp1d()`
3. Create conversion functions to operate on your data sets to accomplish the conversions from time to velocity and from applied cage voltage to electric field and electric field divided by pressure.
4. Create arrays of velocity vs. $E$ and velocity vs. $E/P$.  Then plot your results for the two mixtures on large, easy to read graphs with grids, and include them in your group notebook for discussion.


## Calibration

### Read in your calibration data set

The calibration data shoul be a spreadsheet that has two columns: (1) time between start and stop pulses (in microseconds) and (2) peak positions.  (Peak widths are optional, as they are likely to be very small.)

[**Pandas**](https://pandas.pydata.org/pandas-docs/stable) is a useful library of data manipulation functions.  It has a very easy way to read in data from a spreadsheet and it creates a structure called a `DataFrame`.  You will use this below to read in your spreadsheet data from the pulse-height peak measurements.

In [ ]:
# Execute this cell to include the necessary Pandas functions.
#
from pandas import read_csv, DataFrame

Here is how to use the `read_csv()` function. You should have the spreadsheet file saved as a ".csv" type ("comma separated values").  This is the simplest form of a spreadsheet.  The column labels, e.g., **Time (us)** and **Position (Ch)**, will denote the arrays.  Your labels may be different.

You may obtain the arrays for each column by using the column label, e.g., `Time_cal['Time (us)']` is the array created from that spreadsheet column for the time values.  Below shows how to use the function and how to display the data read in.

In [ ]:
# I assume the file is called 'time_cal.csv'.  
# The structure 'Time_cal' is a Pandas DataFrame

Time_cal = read_csv('time_cal.csv')

# Arrays may be copied from the dataframe
cal_times =  Time_cal['Time (us)']
cal_chans = Time_cal['Position (Ch)']

# Simply stating the DataFrame as the last line prints a nice table:
Time_cal

### Make a line fit & time calibration function

Use LMFit to cerate a line fit to your calibration data set.  Then create a conversion function that allows a channel value to be converted to a time value.

In [ ]:
# You write this.  Use what you learned in previous analysis exercises.



### Read in your drift time measurements

You should create 4 dataframes or array-sets corresponding to the 4 spreadsheets described in the outline.  Repeated application of `read_csv()` will do the trick.

In [ ]:
# You write this



### Apply the time calibrations

Create new arrays that contain the time values corresponding to peak locations and widths for each of the dataframes.  You may add columns to your data frames to hold these transformed arrays.  For example, if your dataframe for the 93/7 mixture for postion 1 is called `Drift93_7_pos1` and the column for the peak positions is labeled `'Position (Ch)'`, and your calibration function is called `chan2time()`, you can make a new column in your dataframe lile this:

<code>
    Drift93_7_pos1['Time (us)'] = chan2time(Drift93_7_pos1['Position (Ch)'])
    # Print out the expanded dataframe
    Drift93_7_pos1
</code>

This will help keep track of your data by putting into a larger structure that it is derived from.

#### CAUTION: Converting peak widths

A peak width is a **difference** of channel numbers, not an absolute value referenced to zero.  So unless your calibration intercept is either zero or very close to it, you should not include it when you convert peaks widths in channel numbers to peak widths in microseconds.

## Data Reduction

### Include physical dimensions and pressure

You now have data sets that have true time values versus accelerating-cage voltages.  Further data reduction requires these absolute time units to be converted to differences in time and then to average speeds by dividing the time difference into the length difference between position 1 and position 2.  

The other conversion to be made is the one from voltage to electric field and electric field divided by gas pressure.  Assume the gas pressure is equal to 1 atmosphere or 760 torr (equals 760 mm-Hg).  We did not measure the pressure, but the value is close enough to allow comparison to published data sets.

These conversions require two physical distances.  The field calculation requires the distance between the end electrode and the wires that create the outer part of the gas detector sub-assembly.  The velocity calculation requires the distance between position 1 and position 2.  You should have measured these with a good ruler or caliper.

In [ ]:
# Enter your measurements

# position 1 - position 2 in cm:
delta_x = # in cm

# distance between HV electrodes and ground electrodes on inside of the cage (that is,
# the electrodes forming the "ground" or "outside" of the wire detector)
Delta_s = # in cm

pressure = 760.0 # standard atmosphere in torr


### Create an interpolation function for position 2

You need to subtract the time measurements taken with the source in position 2 from those taken with the source in position 1.  But if your data points are not taken at exactly the same voltages, you will need to interpolate.  Fortunately the Python libraries have already provided a solution: the **SciPy** `interpolate` package, which you can read about here: [SciPy Interpolate](https://docs.scipy.org/doc/scipy/reference/reference/interpolate.html).

The function to use is `interp1d()`.  Notice that it creates a *function*. Then all you need to do is feed it your list of voltages from position 2 and it will calculate the interpolated times from position 1.  From there it is easy to create an array of time differences. 

In [ ]:
# Read up on SciPy's interpolation package and create an interpolation function



### Calculate the time differences and velocities

Use the function plus the physical constant to calculate the drift velocities.

In [ ]:
# You write this



### Calculate $E$ and $E/P$

You can transform the voltage arrays directly, because they simply scale by constants.  Do that.

In [ ]:
# You write this



### Plot the results

Make two big beautiful plots, one with drift velocity $w$ vs. $E$ and one with $w$ vs. $E/P$.  Then copy them into your group notebook and compare them to other data available from the experiment page. 

In [ ]:
# You write this

